In [ ]:
–ü–∞–π–ø–ª–∞–π–Ω
========
–ß–∞—Å—Ç—å 3.

1. –°–º–æ—Ç—Ä–∏—Ç –ø—É—Ç–∏
2. –ì–æ—Ç–æ–≤–∏—Ç DataFrame –∏–∑ photos.csv
3. –ì–µ–Ω–µ—Ä–∏—Ä—É–µ—Ç geojson —Å –ø–æ–ª—è–º–∏ –∏–∑ DataFrame - 
4. –î–æ–±–∞–≤–ª—è–µ—Ç –∫–æ–¥ —Å—Ç—Ä–∞–Ω—ã —á–µ—Ä–µ–∑ "https://nominatim.openstreetmap.org/reverse"

In [24]:
import pandas as pd
import json
import os

# === üìÅ –ü—É—Ç–∏ –∫ —Ñ–∞–π–ª–∞–º
CSV_PATH = "csv/photos.csv"
GEOJSON_PATH = "geojson/photos.geojson"

# === üóÇÔ∏è –ü—Ä–æ–≤–µ—Ä—è–µ–º, —Å—É—â–µ—Å—Ç–≤—É–µ—Ç –ª–∏ –ø–∞–ø–∫–∞ geojson
if not os.path.exists('geojson'):
    os.makedirs('geojson')

In [26]:
# === üì¶ –ß–∏—Ç–∞–µ–º CSV
df = pd.read_csv(CSV_PATH)

# === üéØ –§–∏–ª—å—Ç—Ä—É–µ–º –¥–∞–Ω–Ω—ã–µ —Å –∫–æ–æ—Ä–¥–∏–Ω–∞—Ç–∞–º–∏
df = df.dropna(subset=['latitude', 'longitude'])

# === üó∫Ô∏è –ì–µ–Ω–µ—Ä–∞—Ü–∏—è GeoJSON
features = []
for _, row in df.iterrows():
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']]
        },
        "properties": {
            "filename": row['filename'],
            "folder": row['folder'],
            "year": row['year'],
            "month": row['month'],
            "day": row['day'],
            "source_type": row['source_type']
        }
    })

In [28]:
# === üíæ –°–æ—Ö—Ä–∞–Ω—è–µ–º –≤ —Ñ–∞–π–ª
with open(GEOJSON_PATH, 'w', encoding='utf-8') as f:
    json.dump({
        "type": "FeatureCollection",
        "features": features
    }, f, ensure_ascii=False, indent=4)

print(f"‚úÖ GeoJSON —É—Å–ø–µ—à–Ω–æ —Å–≥–µ–Ω–µ—Ä–∏—Ä–æ–≤–∞–Ω: {GEOJSON_PATH}")
print(f"üìå –í—Å–µ–≥–æ —Ç–æ—á–µ–∫ –¥–æ–±–∞–≤–ª–µ–Ω–æ: {len(features)}")

‚úÖ GeoJSON —É—Å–ø–µ—à–Ω–æ —Å–≥–µ–Ω–µ—Ä–∏—Ä–æ–≤–∞–Ω: geojson/photos.geojson
üìå –í—Å–µ–≥–æ —Ç–æ—á–µ–∫ –¥–æ–±–∞–≤–ª–µ–Ω–æ: 2431


In [ ]:
# –í–æ—Ç –ø–æ–ª–Ω—ã–π —Å–∫—Ä–∏–ø—Ç, –∫–æ—Ç–æ—Ä—ã–π –±–µ—Ä–µ—Ç —Å—Å—ã–ª–∫–∏ –∏–∑ google_drive_links.csv, 
# –ø–æ–¥—Å—Ç–∞–≤–ª—è–µ—Ç –∏—Ö –≤ photos.geojson –∏ —Å–æ—Ö—Ä–∞–Ω—è–µ—Ç –≤ –Ω–æ–≤—ã–π —Ñ–∞–π–ª photos.geojson.

In [30]:
import pandas as pd
import json

# === üìÅ –ß—Ç–µ–Ω–∏–µ CSV
df_links = pd.read_csv('csv/google_drive_links.csv')
df_links['normalized'] = df_links['filename'].str.lower()

# === üìÅ –ß—Ç–µ–Ω–∏–µ GeoJSON
with open('geojson/photos.geojson', 'r', encoding='utf-8') as file:
    geojson_data = json.load(file)

missing = []

# === –û–±–Ω–æ–≤–ª–µ–Ω–∏–µ
for feature in geojson_data['features']:
    geo_name = feature['properties']['filename'].lower()
    match = df_links[df_links['normalized'] == geo_name]

    if not match.empty:
        row = match.iloc[0]
        feature['properties']['image'] = f"https://drive.google.com/thumbnail?id={row['file_id']}"
        feature['properties']['fullname'] = row['link']
        if pd.notna(row['description']) and row['description'] != 'No Description':
            feature['properties']['description'] = row['description']
    else:
        missing.append(geo_name)

# === üíæ –°–æ—Ö—Ä–∞–Ω–µ–Ω–∏–µ
with open('geojson/photos.geojson', 'w', encoding='utf-8') as file:
    json.dump(geojson_data, file, ensure_ascii=False, indent=4)

print(f"‚úÖ –î–æ–±–∞–≤–ª–µ–Ω—ã —Å—Å—ã–ª–∫–∏ –¥–ª—è: {len(geojson_data['features']) - len(missing)} —Ñ–æ—Ç–æ")
if missing:
    print(f"‚ö†Ô∏è –ù–µ –Ω–∞–π–¥–µ–Ω—ã —Å—Å—ã–ª–∫–∏ –¥–ª—è {len(missing)} —Ñ–æ—Ç–æ:")
    for name in missing:
        print(f" - {name}")

‚úÖ –î–æ–±–∞–≤–ª–µ–Ω—ã —Å—Å—ã–ª–∫–∏ –¥–ª—è: 2431 —Ñ–æ—Ç–æ


In [ ]:
# --- –µ—Å–ª–∏ –Ω—É–∂–Ω–æ –¥–æ–±–∞–≤–∏—Ç—å –≤ json –ø–æ–ª–µ —Å –∫–æ–¥–æ–º —Å—Ç—Ä–∞–Ω—ã

In [33]:
import json
import requests
import time
from tqdm import tqdm
import sys

GEOJSON_FILE = "geojson/photos.geojson"
NOMINATIM_URL = "https://nominatim.openstreetmap.org/reverse"
DELAY = 2  # –ó–∞–¥–µ—Ä–∂–∫–∞ –º–µ–∂–¥—É –∑–∞–ø—Ä–æ—Å–∞–º–∏, —á—Ç–æ–±—ã –∏–∑–±–µ–∂–∞—Ç—å –ª–∏–º–∏—Ç–∞ API

HEADERS = {
    'User-Agent': 'PhotoMapsApp (example@domain.com)'
}

def get_country_code(lat, lon):
    """
    –ü–æ–ª—É—á–µ–Ω–∏–µ –∫–æ–¥–∞ —Å—Ç—Ä–∞–Ω—ã –ø–æ –∫–æ–æ—Ä–¥–∏–Ω–∞—Ç–∞–º —á–µ—Ä–µ–∑ Nominatim API
    """
    try:
        response = requests.get(NOMINATIM_URL, params={
            'format': 'json',
            'lat': lat,
            'lon': lon
        }, headers=HEADERS)
        
        if response.status_code == 200:
            data = response.json()
            country_code = data.get('address', {}).get('country_code', '').upper()
            if country_code:
                return country_code
            else:
                print(f"‚ùå –û—à–∏–±–∫–∞: –°—Ç—Ä–∞–Ω–∞ –Ω–µ –Ω–∞–π–¥–µ–Ω–∞ –¥–ª—è –∫–æ–æ—Ä–¥–∏–Ω–∞—Ç {lat}, {lon}")
                sys.exit(1)
        else:
            print(f"‚ùå –û—à–∏–±–∫–∞ –∑–∞–ø—Ä–æ—Å–∞: —Å—Ç–∞—Ç—É—Å {response.status_code}")
            sys.exit(1)

    except Exception as e:
        print(f"‚ùå –û—à–∏–±–∫–∞ —Å–æ–µ–¥–∏–Ω–µ–Ω–∏—è: {e}")
        sys.exit(1)


def enrich_geojson():
    """
    –û–±–æ–≥–∞—â–µ–Ω–∏–µ GeoJSON –¥–∞–Ω–Ω—ã–º–∏ –æ —Å—Ç—Ä–∞–Ω–µ –ø–æ –∫–æ–æ—Ä–¥–∏–Ω–∞—Ç–∞–º
    """
    with open(GEOJSON_FILE, 'r') as file:
        geojson_data = json.load(file)

    # –ü—Ä–æ–≥—Ä–µ—Å—Å-–±–∞—Ä
    for feature in tqdm(geojson_data["features"], desc="–û–±—Ä–∞–±–æ—Ç–∫–∞ —Ç–æ—á–µ–∫"):
        lat, lon = feature["geometry"]["coordinates"][1], feature["geometry"]["coordinates"][0]
        country_code = get_country_code(lat, lon)
        feature["properties"]["country_code"] = country_code
        time.sleep(DELAY)  # –ü–∞—É–∑–∞, —á—Ç–æ–±—ã –Ω–µ —Å–ª–æ–≤–∏—Ç—å –±–ª–æ–∫–∏—Ä–æ–≤–∫—É API

    with open(GEOJSON_FILE, 'w') as file:
        json.dump(geojson_data, file, indent=4)
        print(f"\n‚úÖ –§–∞–π–ª {GEOJSON_FILE} —É—Å–ø–µ—à–Ω–æ –æ–±–Ω–æ–≤–ª—ë–Ω —Å –∫–æ–¥–∞–º–∏ —Å—Ç—Ä–∞–Ω.")


# === –ó–∞–ø—É—Å–∫ –ø—Ä–æ—Ü–µ—Å—Å–∞ ===
enrich_geojson()

–û–±—Ä–∞–±–æ—Ç–∫–∞ —Ç–æ—á–µ–∫: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 2431/2431 [1:32:26<00:00,  2.28s/it]


‚úÖ –§–∞–π–ª geojson/photos.geojson —É—Å–ø–µ—à–Ω–æ –æ–±–Ω–æ–≤–ª—ë–Ω —Å –∫–æ–¥–∞–º–∏ —Å—Ç—Ä–∞–Ω.
